In [12]:
# scikit-surprise
# https://learn.microsoft.com/en-US/cpp/windows/latest-supported-vc-redist?view=msvc-170#visual-studio-2015-2017-2019-and-2022
# https://stackoverflow.com/questions/61365790/error-could-not-build-wheels-for-scipy-which-use-pep-517-and-cannot-be-installe
# pip install --upgrade pip setuptools wheel / pip3 install scikit-surprise

In [13]:
import os
import sqlalchemy
import pandas as pd
from dotenv import load_dotenv 
from surprise import Reader, Dataset, SVD
from sqlalchemy import create_engine, text
from sklearn.preprocessing import OneHotEncoder
from surprise.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from surprise.model_selection import train_test_split as tts2

In [14]:
load_dotenv()
DB_URL = os.getenv("DB_URL")
engine = create_engine(DB_URL)

try:
    connection = engine.connect()
    print("Successfully connected to the database")
except Exception as e:
    print(f"Failed to connect to the database: {e}")

print(f"SQLAlchemy version: {sqlalchemy.__version__}")

Successfully connected to the database
SQLAlchemy version: 2.0.21


In [15]:
query = text('SELECT * FROM epic_5')
df = pd.read_sql_query(query, connection)
df.head()

,PersoonId,CampagneId,aantal_sessies,aantal_bezoeken,SessieThema,SoortCampagne,TypeCampagne,ThemaDuurzaamheid,ThemaFinancieelFiscaal,ThemaInnovatie,ThemaInternationaalOndernemen,ThemaMobiliteit,ThemaOmgeving,ThemaSalesMarketingCommunicatie,ThemaStrategieEnAlgemeenManagement,ThemaTalent,ThemaWelzijn
0,8BCEBB99-C0C1-E111-A45C-00505680000A,3F797E25-9356-EC11-8F8F-000D3AAAA40E,1,0,Human Resources,Offline,Opleiding,0,1,0,0,1,0,0,1,1,0
1,F29FCEE6-3066-EB11-8119-001DD8B72B61,BE590F6B-2A6E-EC11-8943-000D3A279B65,5,0,Human Resources,On en Offline,Project,1,0,0,0,1,0,0,0,1,1
2,A372F40E-CDB1-E411-9B05-005056B06EB4,870BA609-A856-ED11-BBA2-6045BD895D85,1,3,"Digitalisering, IT & Technologie",Offline,Netwerkevenement,1,0,1,0,0,0,0,0,1,1
3,7F25E74D-58C9-E711-80EC-001DD8B72B62,61FE6D67-F088-E811-80F3-001DD8B72B61,2,0,Human Resources,Offline,Netwerkevenement,0,0,0,0,0,0,0,0,0,0
4,7319F14F-AB67-E111-A00F-00505680000A,289F1F3C-4A67-ED11-9561-6045BD895B5A,1,0,Netwerking,Offline,Netwerkevenement,1,0,1,0,1,1,1,0,0,0


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52567 entries, 0 to 52566
Data columns (total 17 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   PersoonId                           52567 non-null  object
 1   CampagneId                          52567 non-null  object
 2   aantal_sessies                      52567 non-null  int64 
 3   aantal_bezoeken                     52567 non-null  int64 
 4   SessieThema                         52567 non-null  object
 5   SoortCampagne                       52567 non-null  object
 6   TypeCampagne                        52567 non-null  object
 7   ThemaDuurzaamheid                   52567 non-null  int64 
 8   ThemaFinancieelFiscaal              52567 non-null  int64 
 9   ThemaInnovatie                      52567 non-null  int64 
 10  ThemaInternationaalOndernemen       52567 non-null  int64 
 11  ThemaMobiliteit                     52567 non-null  in

In [17]:
# WORK IN PROGRESS UNDER THIS LINE

In [18]:
df_tfidf = df

df_tfidf['CampagneSessieInformatie'] = df_tfidf['SessieThema'] + ' ' + df_tfidf['SoortCampagne'] + ' ' + df_tfidf['TypeCampagne']

df_tfidf = df_tfidf[['PersoonId', 'CampagneId', 'aantal_sessies', 'CampagneSessieInformatie']]

df_tfidf.head()

,PersoonId,CampagneId,aantal_sessies,CampagneSessieInformatie
0,8BCEBB99-C0C1-E111-A45C-00505680000A,3F797E25-9356-EC11-8F8F-000D3AAAA40E,1,Human Resources Offline Opleiding
1,F29FCEE6-3066-EB11-8119-001DD8B72B61,BE590F6B-2A6E-EC11-8943-000D3A279B65,5,Human Resources On en Offline Project
2,A372F40E-CDB1-E411-9B05-005056B06EB4,870BA609-A856-ED11-BBA2-6045BD895D85,1,"Digitalisering, IT & Technologie Offline Netwe..."
3,7F25E74D-58C9-E711-80EC-001DD8B72B62,61FE6D67-F088-E811-80F3-001DD8B72B61,2,Human Resources Offline Netwerkevenement
4,7319F14F-AB67-E111-A00F-00505680000A,289F1F3C-4A67-ED11-9561-6045BD895B5A,1,Netwerking Offline Netwerkevenement


In [ ]:
# VANAF HIER ONZEKER

In [19]:
vectorizer = TfidfVectorizer()

campagne_sessie_matrix = vectorizer.fit_transform(df_tfidf['CampagneSessieInformatie'])

In [20]:
from sklearn.decomposition import TruncatedSVD

# Assuming tfidf_matrix is your TF-IDF matrix
svd = TruncatedSVD(n_components=40)  # Reduce to 100 dimensions
reduced_campagne_sessie_matrix = svd.fit_transform(campagne_sessie_matrix)

In [21]:
content_model = cosine_similarity(reduced_campagne_sessie_matrix)

In [22]:
# WORK IN PROGRESS ABOVE THIS LINE

In [23]:
df_svd = df[['PersoonId', 'CampagneId', 'aantal_sessies']]
df_svd.shape

(52567, 3)

In [24]:
min_aantal_sessies = df_svd.loc[df_svd['aantal_sessies'] > 2]
min_aantal_sessies.shape

(6512, 3)

In [25]:
min_aantal_sessies['aantal_sessies'].value_counts()

3     1372
4     1064
6     1048
5      837
7      466
10     307
11     274
14     212
13     195
12     185
8      183
9      149
15     125
17      58
18      14
16      14
26       2
20       2
27       2
22       2
24       1
Name: aantal_sessies, dtype: int64

In [26]:
df_svd = min_aantal_sessies

In [27]:
train_set, test_set = train_test_split(df_svd, test_size=0.25, random_state=7)

In [28]:
reader = Reader(rating_scale=(df_svd['aantal_sessies'].min(), df_svd['aantal_sessies'].max()))

df_svd_surprise = Dataset.load_from_df(df_svd[['PersoonId', 'CampagneId', 'aantal_sessies']], reader)

df_svd_train = Dataset.load_from_df(train_set[['PersoonId', 'CampagneId', 'aantal_sessies']], reader)

In [29]:
param_grid = {"n_epochs": [5, 10, 15], "lr_all": [0.02, 0.025, 0.03], "reg_all": [0.1, 0.2, 0.4], "n_factors": [5, 15, 25]}
grid_search_SVD = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

grid_search_SVD.fit(df_svd_train)

print('Best RMSE:', grid_search_SVD.best_score['rmse'], grid_search_SVD.best_params['rmse'])
print('Best MAE:', grid_search_SVD.best_score['mae'], grid_search_SVD.best_params['mae'])

Best RMSE: 1.5468427150762205 {'n_epochs': 15, 'lr_all': 0.03, 'reg_all': 0.1, 'n_factors': 5}
Best MAE: 0.9987649002029674 {'n_epochs': 15, 'lr_all': 0.03, 'reg_all': 0.1, 'n_factors': 5}


In [30]:
trainset, testset = tts2(df_svd_surprise, test_size=0.25, random_state=7)

model_SVD = grid_search_SVD.best_estimator['rmse']

model_SVD.fit(trainset)

predictions = model_SVD.test(testset)

ResultCatcher = pd.DataFrame(predictions, columns=['PersoonId', 'CampagneId', 'Real_Rating', 'Estimated_Rating', 'details'])
ResultCatcher.drop(['details'], axis=1, inplace=True)

In [31]:
ResultCatcher['Estimation_Rounded'] = ResultCatcher.apply(lambda row: round(row.Estimated_Rating), axis=1)
ResultCatcher['Error'] = abs(ResultCatcher['Real_Rating'] - ResultCatcher['Estimation_Rounded'])
ResultCatcher.head()

,PersoonId,CampagneId,Real_Rating,Estimated_Rating,Estimation_Rounded,Error
0,E2FDF3CE-FFF7-E511-8668-005056B06EC4,E4A5EB7F-BA4E-E811-80F0-001DD8B72B62,7.0,7.127586,7,0.0
1,FABB23D1-5D8A-EC11-93B0-6045BD91D362,DF1B73C0-436E-EC11-8943-000D3A279FAC,9.0,8.564590,9,0.0
2,613C4408-1826-E511-93F3-005056B06EC4,19C23572-4E56-EB11-8117-001DD8B72B61,5.0,4.950631,5,0.0
3,AAAC1209-2ED5-E911-8105-001DD8B72B61,C6D12490-8493-E911-80FF-001DD8B72B61,3.0,4.010796,4,1.0
4,ED13F7AE-C829-E311-8A47-005056B06EC4,BE590F6B-2A6E-EC11-8943-000D3A279B65,5.0,5.356286,5,0.0


In [32]:
sum_of_real_rating = ResultCatcher['Real_Rating'].sum()
sum_of_error = ResultCatcher['Error'].sum()
count_of_real_rating = ResultCatcher['Real_Rating'].count()
count_of_correct_predictions = ResultCatcher['Real_Rating'][ResultCatcher['Error'] == 0].count()

error_percentage = (sum_of_error / sum_of_real_rating) * 100
accuracy_percentage = (count_of_correct_predictions / count_of_real_rating) * 100

print("Sum of Real_Rating: ", sum_of_real_rating)
print("Sum of Error: ", sum_of_error)
print("Count of Real_Rating: ", count_of_real_rating)
print("Count of Real_Rating where Error is 0: ", count_of_correct_predictions)

print("Error Percentage: {:.2f}%".format(error_percentage))
print("Accuracy Percentage: {:.2f}%".format(accuracy_percentage))

Sum of Real_Rating:  10583.0
Sum of Error:  1317.0
Count of Real_Rating:  1628
Count of Real_Rating where Error is 0:  778
Error Percentage: 12.44%
Accuracy Percentage: 47.79%


In [33]:
# https://github.com/EmirKorkutUnal/Python-Surprise-Predictions-on-Custom-Dataset

In [34]:
# WORK IN PROGRESS UNDER THIS LINE

In [ ]:
# beide modellen combineren, content based eerst dan collaborative filtering

In [ ]:
# functie maken die voor een campagne de top 10 personen geeft die het meest waarschijnlijk zijn om de campagne te volgen,
# enkel als de persoon de campagne nog niet heeft gevolgd

In [ ]:
# WORK IN PROGRESS ABOVE THIS LINE